# Update location information
### Twitter API key list
#### - Consumer Keys
- API Key & API Secret
#### - Authentication Tokens
- Bearer Token
- Access Token & Access Secret

Read .env file to read API key (Create your own .env file and write down the API keys)

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
BEARER_TOKEN = os.environ.get("BEARER_TOKEN")
API_KEY = os.environ.get("API_KEY")
API_SECRET = os.environ.get("API_SECRET")
ACCESS_TOKEN = os.environ.get("ACCESS_TOKEN")
ACCESS_TOKEN_SECRET = os.environ.get("ACCESS_TOKEN_SECRET")

ModuleNotFoundError: No module named 'dotenv'

Request location information regarding place_id

In [5]:
import requests

def get_data(BEARER_TOKEN, url):
    headers = {'Authorization': "Bearer " + str(BEARER_TOKEN) }
    response = requests.get(url, headers=headers)
    response_data = response.json()
    print(response_data)
    return response_data

Example of requesting information

In [63]:
place_id = "0073b76548e5984f"
get_data(BEARER_TOKEN, f"https://api.twitter.com/1.1/geo/id/{place_id}.json")

{'errors': [{'message': 'Rate limit exceeded', 'code': 88}]}

### Update information to csv

In [38]:
import pandas as pd
import os

def update_location_info(place_id):
    path = "../data/curated/place_id.csv"
    if os.path.exists(path):
        df = pd.read_csv(path, index_col=0)
        if place_id not in df.index:
            tmp = get_data(BEARER_TOKEN, f"https://api.twitter.com/1.1/geo/id/{place_id}.json")
            add = pd.DataFrame({'full_name':[tmp["full_name"]], 'country_code':[tmp['country_code']]}, index=[place_id])
            df = pd.concat([df, add])
            df.to_csv(path)
    else:
        tmp = get_data(BEARER_TOKEN, f"https://api.twitter.com/1.1/geo/id/{place_id}.json")
        add = pd.DataFrame({'full_name':[tmp["full_name"]], 'country_code':[tmp['country_code']]}, index=[place_id])
        add.to_csv(path)

In [61]:
place_id = "0118c71c0ed41109"
update_location_info(place_id)

In [64]:
path = "../data/curated/place_id.csv"
pd.read_csv(path, index_col=0)

,full_name,country_code
0073b76548e5984f,"Sydney, New South Wales",AU
002891d33ed032d3,"Darwin, Northern Territory",AU
0ec0c4fcacbd0083,"Victoria, Australia",AU
0118c71c0ed41109,"Perth, Western Australia",AU
00637ebe39362fcb,"Nowra - Bomaderry, New South Wales",AU
...,...,...
05f5a0d89e34b4f8,"Rangeville, Toowoomba",AU
0116fcf600926d4f,"Unayzah, Kingdom of Saudi Arabia",SA
01a70ec3689d6ca5,"Coraki, New South Wales",AU
5f55ccd2b14ae80b,"Coleraine, Victoria",AU


In [ ]:
import json

path = "../data/raw/sample_tweets.json"
data = None

with open(path, 'r') as f:
  data = json.load(f)
for d in data['listoftweets']:
  if 'place_id' in d['doc']['data']['geo']:
    place_id = d['doc']['data']['geo']['place_id']
    update_location_info(place_id)